In [1]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt
import re
import ipyleaflet as ipy 
import json
import ipywidgets

In [2]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

In [3]:
#landuse = gpd.read_file(wd+\"data//")
acl = gpd.read_file(wd+"data/fulljurisdiction/austincitylimitsgj.geojson")
cbg = gpd.read_file(wd+"data/censusblockgroups/cbggj.geojson")
pop = pd.read_excel(wd+"data/censustables/ACS_17_5yr_totalpop_allcounties.xlsx")
race = pd.read_excel(wd+"data/censustables/ACS_17_5yr_race_includinglatinx.xlsx")
halfbuff = gpd.read_file(wd+"data/coaparks_buffer/halfbuffgj.geojson")
quarbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuffgj.geojson")
varibuff = gpd.read_file(wd+"data/coaparks_buffer/varibuffgj.geojson")
parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
#parks = gpd.read_file("https://data.austintexas.gov/api/geospatial/8f2b-a4q5?method=export&format=GeoJSON")

In [34]:
acl.head()

,city_name,jurisdicti,jurisdic_1,jurisdic_2,jurisdic_3,modified_f,objectid,shape_area,shape_leng,geometry
0,CITY OF AUSTIN,AUSTIN FULL PURPOSE,FULL,FULL PURPOSE,400630281.0,210.0,496.0,7.162866e+06,1.302898e+04,(POLYGON ((-97.6280281276115 30.35433236263618...
1,CITY OF AUSTIN,AUSTIN FULL PURPOSE,FULL,FULL PURPOSE,400455693.0,210.0,261.0,7.167226e+06,1.219104e+04,(POLYGON ((-97.71292748043328 30.1885706379028...
2,CITY OF AUSTIN,AUSTIN FULL PURPOSE,FULL,FULL PURPOSE,34.0,210.0,110.0,7.752723e+09,2.049691e+06,(POLYGON ((-97.83299919290266 30.1495236668398...
3,CITY OF AUSTIN,AUSTIN FULL PURPOSE,FULL,FULL PURPOSE,400455695.0,210.0,167.0,5.040355e+07,6.169732e+04,(POLYGON ((-97.61329293634175 30.1465468341990...


In [31]:
m = ipy.Map(center=(30.26,-97.74), zoom = 10, basemap= ipy.basemaps.Esri.WorldTopoMap)

In [39]:

quarbuf_data = ipy.GeoData(geo_dataframe = quarbuff,
                       style={'color': 'black', 'fillColor': '#eb0c0c', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Quarter Mile Buffer')
#m.add_layer(quarbuf_data)

halfbuf_data = ipy.GeoData(geo_dataframe = halfbuff,
                       style={'color': 'black', 'fillColor': '#ffaa00', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Half Mile Buffer')
#m.add_layer(halfbuf_data)

varibuf_data = ipy.GeoData(geo_dataframe = varibuff,
                       style={'color': 'black', 'fillColor': '#fcf800', 'opacity':0.5, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Variable Mile Buffer')
#m.add_layer(varibuf_data)

park_data = ipy.GeoData(geo_dataframe = parks,
                       style={'color': 'black', 'fillColor': '#5c8000', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':0.6},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'Parks'
                       )

#acl_data = ipy.GeoData(geo_dataframe = acl,
                      # style={'color': 'white', 'opacity':0.05, 'weight':5, 'dashArray':'2', 'fillOpacity':0.6},
                       #name = 'Austin City Limits')
#m.add_layer(acl_data)
list_buffs=[quarbuf_data, halfbuf_data, varibuf_data]

label_parks = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_name = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
label_buff = ipywidgets.Label(layout=ipywidgets.Layout(width='100%'))
def hover_handler(event=None, id=None, properties=None, feature=None):
    label_name.value='Park: '+ str(properties['LOCATION_N'])
    label_parks.value = 'Park acreage: ' + str(properties['ASSET_SIZE'])
    label_buff='Buff'+str(dropdown.value)
    #label_acl.value=str(properties['jurisdicti'])

def hover_handler2(event=None, id=None, properties=None, feature=None):
    label_buff = 'Buff'+str(dropdown.value)

def on_dropdown_change(change):
    if change.new == 'Quarter Mile Buffer':
        m.remove_layer(quarbuf_data)
        quarbuf_data.on_hover(hover_handler2)
        
        m.add_layer(quarbuf_data)
        print(m.layers)
        

dropdown = ipywidgets.Dropdown(options=['Quarter Mile Buffer', 'Half Mile Buffer', 'Varied Buffers'],
                    value='Half Mile Buffer', 
                    description='Radius within Parks',
                    disabled=False)
display(dropdown)
dropdown.observe(on_dropdown_change)

park_data.on_hover(hover_handler)
quarbuf_data.on_hover(hover_handler2)
#acl_data.on_hover(hover_handler_acl)
#m.add_layer(acl_data)
m.add_layer(quarbuf_data)
m.add_layer(park_data)
m.add_control(ipy.LayersControl())
m
ipywidgets.VBox([m, label_name, label_parks, label_buff])

Dropdown(description='Radius within Parks', index=1, options=('Quarter Mile Buffer', 'Half Mile Buffer', 'Vari…

In [20]:
help(ipywidgets.Dropdown)

Help on class Dropdown in module ipywidgets.widgets.widget_selection:

class Dropdown(_Selection)
 |  Dropdown(*args, **kwargs)
 |  
 |  Allows you to select a single item from a dropdown.
 |  
 |  Parameters
 |  ----------
 |  options: list
 |      The options for the dropdown. This can either be a list of values, e.g.
 |      ``['Galileo', 'Brahe', 'Hubble']`` or ``[0, 1, 2]``, or a list of
 |      (label, value) pairs, e.g.
 |      ``[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)]``.
 |  
 |  index: int
 |      The index of the current selection.
 |  
 |  value: any
 |      The value of the current selection. When programmatically setting the
 |      value, a reverse lookup is performed among the options to check that
 |      the value is valid. The reverse lookup uses the equality operator by
 |      default, but another predicate may be provided via the ``equals``
 |      keyword argument. For example, when dealing with numpy arrays, one may
 |      set ``equals=np.array_equal``.
 |

In [37]:
quarbuff.head()

,id,geometry
0,0,(POLYGON ((-97.78882039869158 30.2836917877253...
1,1,(POLYGON ((-97.71370284547962 30.3374071106376...
2,2,(POLYGON ((-97.75667890586269 30.2826743690636...
3,3,(POLYGON ((-97.6515179388645 30.39377256371783...
4,4,(POLYGON ((-97.61381779212277 30.3015054122357...
